# 15.780 Fall 2025
## The Analytics of Operations Management
### Problem Set 4 - Linear Optimization and Supply Chain Optimization
#### Due Date:  Friday 11/21 at 11:59pm EST
---
Name of Student: Sasha Krigel

MIT ID Number: 943271028

---

### Instructions:

1) Submit solutions that are your own, in your own words. You are allowed to discuss with other students in general terms, but make sure you are not copying verbatim from another student. Therefore do not read other students' solutions. If you use material from outside this class, reference it in your solution. 

2) Please download the python file attached in the assignment and complete your answers there in the same file. Read the questions carefully, and make sure you answer every part that the question asks.

3) Include relevant code in the PDF submission even if the question doesn't explicitly ask for it. Upload your solutions as a PDF file. Include your name and MIT ID on the first page.

4) To convert to pdf, you can use the "print to pdf" option in jupyter (or equivalent options in other IDE). There are other options to directly download in to pdf format which might include additional installation of packages. 

5) Show your work and explain your conclusions clearly and precisely. Plots should have clear titles and axis labels so that it is clear what your analysis is showing.

--------------------------------------------------------------------------------------------------------------------------------

In [1]:
# RUN THIS BEFORE STARTING: importing gurobipy
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


---
### **Problem 1: Product Mix and Profit Maximization (30 points)**

A canning company operates two plants (South and Central). Its suppliers provide fresh fruit of three different kinds (peaches, pears, and pineapples) for the company to can. The fruit costs of peaches, pears, and pineapples per unit are 120, 90, and 80 USD respectively. The shipping costs per unit are different for each plant. To the South plant, the shipping costs (respectively, for peaches, pears, and pineapples) are 30, 25, and 60 USD, whereas for the Central plant the shipping costs are 35, 30, and 40 USD. Furthermore, the two plants have different capacities, as the South plant has a capacity of 480 units and the Central plant has a capacity of 550 units. Lastly, the two plants face different labor costs per unit: the South plant's labor costs are 250 USD per unit and the Central plant's labor costs are 200 USD per unit. Labor costs at both plants are independent of fruit type.

Cans of peaches, pears, and pineapples are then sold by the company to distributors at 540, 500, and 520 USD per unit respectively.

The canning company wants to know how much of each fruit they should can at each of the factories in order to maximize its profit. Answer the following questions in Markdown (mathematical expressions can be written in LaTeX; refer to Recitation and LaTeX resources):

&emsp; **a) What decision variables would you use to find the optimal shipments?** (10 points)

$$ x_{p,f}= \text{number of units of fruit p producted on plant p }\\
\text{where }p\in \{South, Center\}, f\in\{Pineapple, Pear, Peach\}$$

&emsp; **b) How would you encode the objective of this LP?** (10 points)


Maximize profit: Sum of total revenue - Sum of total cost
- Revenue = number of units of each fruit * price of each fruit
- Cost = number of units of each fruit * shipping cost of each fruit + number of units of each fruit * labor cost of each fruit



Parameters:
- $C_{f}$: Cost per unit of fruit f
- $S_{p,f}$: Shipping cost of fruit f from plant p
- $L_{p}$: Cost of labor per unit at plant p


$$ \max  \sum_{p\in\{South, Center\}} \sum_{f\in\{peach, pear, pineapple\}} C_{f}\cdot X_{p,f}-   S_{p,f}\cdot X_{p,f} -  \sum_{p\in\{South, Center\}} \sum_{f\in\{peach, pear, pineapple\}} L_{p}\cdot X_{p,f} $$


&emsp; **c) What kind of constraints will this LP require to solve this question? First, describe each constraint in words, then formulate them with mathematical notations (either with LaTeX or via a written description such as "sum over all {variables of this type} over {indices}..." -- in the latter case, make sure to be as precise as possible!)** (10 points)

Constraints:
- Capacity Constraints: we can't produce more at factory than we have capacity for
- Non-negative constraints: we can't produce a negative amount of fruit

1. South produces at most 480 units
$$ \sum_{f\in\{peach, pear, pineapple\}} x_{South, f} \leq 480 $$

2. Central produces at most 550 units
$$ \sum_{f\in\{peach, pear, pineapple\}} x_{Central, f} \leq 550 $$

3. Non-negative constraints
$$ x_{South, f} \geq 0 \quad \forall f \in \{peach, pear, pineapple\}$$
$$ x_{Central, f} \geq 0 \quad \forall f \in \{peach, pear, pineapple\}$$



---
### **Problem 2: Diet Optimization (25 points)**

In this problem, we are given different types of food that each come with a (financial) cost and with nutritional information (amount of calories, protein, fat, and sodium). In addition, we are told the lower and upper limits of what our total nutritional intake must be within for each of the nutrients (e.g., at least 5g of fat but not more than 60g). 

The following code cell contains the data for the problem.

In [8]:
# Data for Problem 2: DO NOT EDIT

# Data 1 : The below dictionary shows the minimum and maximum ranges of each nutrient
categories, minNutrition, maxNutrition = gp.multidict({
    'calories': [2000, 2500],
    'protein':  [80, gp.GRB.INFINITY],
    'fat':      [5, 60], 
    'sodium':   [100, 2000]})

# Data 2 : The below dictionary shows the cost of each available food
foods, cost = gp.multidict({
    'hamburger': 2.69,
    'chicken':   2.89,
    'hot dog':   1.49,
    'fries':     2.69,
    'macaroni':  2.29,
    'pizza':     2.99,
    'salad':     2.49,
    'milk':      0.89,
    'ice cream': 1.99})

# Data 3 : The below dictionary shows the the amount of distinct nutrition available in each food
nutritionValues = {
    ('hamburger', 'calories'): 910,
    ('hamburger', 'protein'):  40,
    ('hamburger', 'fat'):      26,
    ('hamburger', 'sodium'):   730,
    ('chicken',   'calories'): 420,
    ('chicken',   'protein'):  35,
    ('chicken',   'fat'):      10,
    ('chicken',   'sodium'):   990,
    ('hot dog',   'calories'): 560,
    ('hot dog',   'protein'):  24,
    ('hot dog',   'fat'):      32,
    ('hot dog',   'sodium'):   1800,
    ('fries',     'calories'): 380,
    ('fries',     'protein'):  4,
    ('fries',     'fat'):      19,
    ('fries',     'sodium'):   270,
    ('macaroni',  'calories'): 320,
    ('macaroni',  'protein'):  8,
    ('macaroni',  'fat'):      10,
    ('macaroni',  'sodium'):   930,
    ('pizza',     'calories'): 820,
    ('pizza',     'protein'):  10,
    ('pizza',     'fat'):      82,
    ('pizza',     'sodium'):   820,
    ('salad',     'calories'): 240,
    ('salad',     'protein'):  24,
    ('salad',     'fat'):      12,
    ('salad',     'sodium'):   680,
    ('milk',      'calories'): 100,
    ('milk',      'protein'):  12,
    ('milk',      'fat'):      2.5,
    ('milk',      'sodium'):   125,
    ('ice cream', 'calories'): 830,
    ('ice cream', 'protein'):  8,
    ('ice cream', 'fat'):      100,
    ('ice cream', 'sodium'):   180}

First, we construct a model that aims to minimize the financial cost of the food items we consume whilst guaranteeing that we are within the limits of calories, protein, fat, and sodium (which has already been implemented for you below). Before moving on, see if you can first write the mathematical formulation for this problem yourself. We will not expect you to turn in your mathematical formulation, but nonetheless it is always good practice to write out model formulations first before building them in code.

Variables:

$$ X_{f}= \text{number each food units of food fwe buy } \forall f\in foods $$

Minimize cost of food bought:

$$ \min  \sum_{f\in foods} cost[f] * X_{f} $$

Subject to nutritional constraints:

$$ minNutrition[c] \leq \sum_{ f\in foods} nutritionValues[f][c]* X_{f} \leq maxNutrition[c] \quad \forall c \in \{categories\}$$







In [9]:
# First, we define a variable that contains the optimization model
m = gp.Model("diet1")

# Second, we add decision variables to the model that capture for each food item the quantity bought
buy = {}
for f in foods:
    buy[f] = m.addVar(name=f)

# Third, we set our objective: minimizing the cost of the food
m.setObjective(sum(buy[f]*cost[f] for f in foods), gp.GRB.MINIMIZE)

# Fourth, we set constraints that enforce for each nutrition category (calories, protein, fat, sodium) that we be in the feasible range
for c in categories:
    m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods), # this sum captures the amount of each nutrient
               minNutrition[c], # this is the lower bound (from the data)
               maxNutrition[c], # this is the upper bound (from the data)
               c) # this is the name for the constraint; a name is not required but it is a good habit to give each variable/constraint a unique name
    
# We could have written the constraints also by writing
# m.addConstr(sum(nutritionValues[f, c] * buy[f] for f in foods) >= minNutrition[c])
# and
# m.addConstr(sum(nutritionValues[f, c] * buy[f] for f in foods) <= maxNutrition[c])


Once our model has been written in code, we can now tell Gurobi to find an optimal solution. When Gurobi solves, it gives us a lot of information that we may not care about (at least, not for right now). We define a function that ends up printing the interesting bits of the optimal solution for us in the next code cell.

In [10]:
def printSolution(m):
    if m.status == gp.GRB.OPTIMAL: # to check whether an optimal solution was found
        buyx = m.getAttr('x', buy)
        print('Cost: %g' %(sum(cost[f] * buyx[f] for f in foods)))
        print('\nNutrients:')
        for c in categories:
            print('%s %g'%(c, sum(buyx[f]*nutritionValues[f,c] for f in foods)))
        print('\nBuy:')
        for f in foods:
            if buy[f].x > 0.0001:
                print('%s %g' % (f, buyx[f]))
        
    else:
        print('No solution')

We can then run the optimizer and print out our solution.

In [11]:
# Now we solve, and get a bunch of info we don't necessarily understand
# If we want to stop gurobi from printing that info wee can add m.setParam( 'OutputFlag', False ) to the code to stop Gurobi from printing
m.optimize()
# Now we print the interesting aspects of our solution:
printSolution(m)

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.2.0 24C2101)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 4 rows, 12 columns and 39 nonzeros (Min)
Model fingerprint: 0xe015acc8
Model has 9 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+01, 2e+03]
Presolve time: 0.00s
Presolved: 4 rows, 12 columns, 39 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.462500e+02   0.000000e+00      0s
       5    5.8825549e+00   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.882554945e+00
Cost: 5.88255

Nutrients:
calories 2000
protein 87.8022
fat 60
sodium 1845.6

Buy:
hamburger 2.08791
hot dog 0.178571


Now, we'll answer some questions and think about the problem from a different optimization perspective.

&emsp; **a) Rather than aiming to minimize cost, we want to design a diet to build muscle (i.e., we need to maximize our protein intake). 
The cell below has most of the protein maximization model written out (same as above), with only the objective missing. Formulate and add the new objective to the optimization problem below and have Gurobi solve it. Comment on the results.** (10 points)

Variables:

$$ X_{f}= \text{number each food units of food fwe buy } \forall f\in foods $$

Maximize protein across all food bought:

$$ \max  \sum_{f\in foods} nutritionValue[f]["protein"] * X_{f} $$

Subject to nutritional constraints:

$$ minNutrition[c] \leq \sum_{ f\in foods} nutritionValues[f][c]* X_{f} \leq maxNutrition[c] \quad \forall c \in \{categories\}$$

$$ X_{f} \geq 0 \quad \forall f \in \{foods\}$$

In [12]:
# First, we define a variable that contains the optimization model
m = gp.Model("diet2")

# Second, we add decision variables to the model that capture for each food item the quantity bought
buy = {}
for f in foods:
    buy[f] = m.addVar(name=f)

# Third, we set our objective: maximizing the amount of protein in our diet.
##############################
m.setObjective(sum(buy[f]*nutritionValues[(f,'protein')] for f in foods), gp.GRB.MAXIMIZE)

##############################

# Fourth, we set constraints that enforce for each nutrition category (calories, protein, fat, sodium) that we be in the feasible range
for c in categories: 
    m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods), # again: amount of each nutrient
               minNutrition[c], # again: lower bound (from the data)
               maxNutrition[c], # again: upper bound (from the data)
               c) # name for constraint
m.optimize()
printSolution(m)

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.2.0 24C2101)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 4 rows, 12 columns and 39 nonzeros (Max)
Model fingerprint: 0x5c75550f
Model has 9 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [4e+00, 4e+01]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+01, 2e+03]
Presolve time: 0.01s
Presolved: 4 rows, 12 columns, 39 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1093750e+30   1.015527e+31   4.109375e+00      0s
       3    1.5957706e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.595770567e+02
Cost: 11.6601

Nutrients:
calories 2000
protein 159.577
fat 60
sodium 2000

Buy:
hamburger 1.05227
milk 9.73515
ice cream 0.0830297


### Results

We can consume a maximum of 159.577g of protein before reaching nutritional upper bounds on sodium and fat (2000mg and 100g, respectively). We need to consume that amount of fat in order to meet our minimum caloric demands, as the lower constraint on calroies was also binding. It seems that the combination of foods that maximimizes protein content while also meeting the lower bound on calories is hamburgers, milk, and ice cream. Since hamburgers has the highest protein content and milk has a good protein to cost ratio, we buy as much as possible.


&emsp; **b) Similar to part (2a), we still want to maximize our protein intake. However, we now have a budget constraint: our total cost can not exceed 11. Write out and solve this new optimization problem (you should be able to copy-paste the cell from question A and add just one line with the new budget constraint). Comment on the results.** (10 points)

Variables:

$$ X_{f}= \text{number each food units of food fwe buy } \forall f\in foods $$

Maximize protein across all food bought:

$$ \max  \sum_{f\in foods} nutritionValue[f]["protein"] * X_{f} $$

Subject to nutritional constraints:

$$ minNutrition[c] \leq \sum_{ f\in foods} nutritionValues[f][c]* X_{f} \leq maxNutrition[c] \quad \forall c \in \{categories\}$$
$$ \sum_{ f\in foods} cost[f]* X_{f} \leq 11 $$

$$ X_{f} \geq 0 \quad \forall f \in \{foods\}$$



In [13]:
# First, we define a variable that contains the optimization model
m = gp.Model("diet3")

# Second, we add decision variables to the model that capture for each food item the quantity bought
buy = {}
for f in foods:
    buy[f] = m.addVar(name=f)

# Third, we set our objective as in part A: maximizing the amount of protein in our diet.
##############################
# TODO: write your code here (enter the objective function below)
m.setObjective(sum(buy[f]*nutritionValues[(f,'protein')] for f in foods), gp.GRB.MAXIMIZE)
##############################

# Fourth, we set constraints that enforce for each nutrition category (calories, protein, fat, sodium) that we be in the feasible range
for c in categories: # 
    m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods), # again: amount of each nutrient
               minNutrition[c], # this is the lower bound (from the data)
               maxNutrition[c], # this is the upper bound (from the data)
               c) # this is the name for the constraint


# Finally, you need to add a constraint that bounds the cost of items bought by 11
##############################
m.addRange(sum(cost[f] * buy[f] for f in foods), 0, 11)

##############################                                                 
    
m.optimize()
printSolution(m)

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.2.0 24C2101)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 5 rows, 13 columns and 49 nonzeros (Max)
Model fingerprint: 0x46f02e34
Model has 9 linear objective coefficients
Coefficient statistics:
  Matrix range     [9e-01, 2e+03]
  Objective range  [4e+00, 4e+01]
  Bounds range     [1e+01, 2e+03]
  RHS range        [1e+01, 2e+03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s
Presolved: 5 rows, 12 columns, 48 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+02   3.275022e+02   0.000000e+00      0s
       5    1.5313447e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.531344712e+02
Cost: 11

Nutrients:
calories 2021.22
protein 153.134
fat 54.7296
sodium 2000

Buy:
hamburger 1.29207
milk 8.4543


### **Results**

With the added cost constraint, the optimal solution (maximum protein consumption) is lower, having been reduced frorm 159mg to 153.123. The constraining nutrient is still sodium (as that constrain is binding), but fat is no longer binding since we can't afford to buy as much ice cream (high in fat) with the new cost constraint. Our total cost is now exactly $11, so the factors limiting protein intake appear to be both cost and sodium

&emsp; **c) Explain in words why the protein amount (objective value) we get is lower in part (2b) than it was in part (2a). Can you identify the minimum budget we need to consume the maximum amount of protein? (either implement this, or just explain how you would do it).** (5 points)

The maximum protein amount is lower because the the amount of high-protein food we can buy is constrained by the upper bound on cost. In both cases, the bottlenecking factor is the sodium constraint, where the upper bound of 2000mg is reached in both scenarios. This implies that the lack of the cost constraint in the first scenario allows us to buy more expensive foods with lower sodium content to up our protein intake without exceeding sodium limits. In the scenario with the cost constraint, we are forced to buy less expensive foods with higher sodium content to stay within the sodium limit and can get less protein.

Below, I have implemented a model that addresses the problem of finding the minimum cost diet that reaches the maximum protein intake from the unconstrained protein maximization problem. The model satisfies the nutritional requirements while minimizing the cost. The minimum cost to get 159.577g of protein is $11.6601.
<!-- categories, minNutrition, maxNutrition = gp.multidict({
    'calories': [2000, 2500],
    'protein':  [80, gp.GRB.INFINITY],
    'fat':      [5, 60], 
    'sodium':   [100, 2000]})


## NO cost constraint
Cost: 11.6601

Nutrients:
calories 2000
protein 159.577
fat 60
sodium 2000

Buy:
hamburger 1.05227
milk 9.73515
ice cream 0.0830297

## COST constraint

Cost: 11

Nutrients:
calories 2021.22
protein 153.134
fat 54.7296
sodium 2000

Buy:
hamburger 1.29207
milk 8.4543


The cost constraint limits the  -->

In [ ]:
# OPTIONAL: implement your code here (hint: copy the code cell from part 2b and make a few changes/additions)
MAX_PROTEIN = 159.577
# We can try to minimize cost while adding an extra constrain that the total protein intake must equal maximum found in the previous part of the problem
m = gp.Model("diet4")

# Second, we add decision variables to the model that capture for each food item the quantity bought
buy = {}
for f in foods:
    buy[f] = m.addVar(name=f)

# Third, we set our objective as in part A: maximizing the amount of protein in our diet.
##############################
m.setObjective(sum(buy[f]*cost[f] for f in foods), gp.GRB.MINIMIZE)

##############################

# Fourth, we set constraints that enforce for each nutrition category (calories, protein, fat, sodium) that we be in the feasible range
for c in categories: # 
    m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods), # again: amount of each nutrient
               minNutrition[c], # this is the lower bound (from the data)
               maxNutrition[c], # this is the upper bound (from the data)
               c) # this is the name for the constraint


# Finally, you need to add a constraint that bounds the cost of items bought by 11
##############################
m.addRange(sum(buy[f]*nutritionValues[(f, "protein")] for f in foods), MAX_PROTEIN, gp.GRB.INFINITY)

##############################                                                 
    
m.optimize()
printSolution(m)


Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.2.0 24C2101)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 5 rows, 12 columns and 48 nonzeros (Min)
Model fingerprint: 0x89039bfc
Model has 9 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+01, 2e+03]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 4 rows, 12 columns, 39 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.061442e+02   0.000000e+00      0s
       3    1.1660128e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.166012758e+01
Cost: 11.6601

Nutrients:
calories 2000
protein 159.577
fat 59.9999
sodium 2000

Buy:
hamburger 1.05228
milk 9.73514
ice cre

---
### **Problem 3: Factory Production Schedule (25 points)**

In this question, we need to decide on a production schedule for a factory. The factory is tasked with producing 3 different goods, and it has 3 different machines. Each machine can be tasked to work on any of the goods, but certain machines are faster at producing certain goods than they are at others. Our goal is to minimize the time until all our orders are fulfilled. **What is the optimal production schedule, and what is the shortest amount of time required to fulfill all orders?** Formulate and solve the optimization problem in Gurobi.

(HINT: In setting up your optimization problem it will be useful to define 10 different variables: 9 for each machine/product combination pairing (i.e., "How many hours does machine `i` spend on product `j`?") and 1 for the objective (i.e., "What is the maximum time any of the machines are working?") which we refer to as makespan.)

In the cell below we provide you with the data for the problem:

In [15]:
# Data for Problem 3: DO NOT EDIT

# we have products 'p1', 'p2', 'p3', and we need to produce (respectively) 100, 200, 400 units of each
product_demand = {'p1': 100, 'p2': 200, 'p3': 400}
machines = ['m1', 'm2', 'm3']
# we have three machines, 'm1', 'm2', and 'm3' and the number of products of each type that can be produced by each of the machines is given in the dictionary below
machine_speed = {('m1','p1'):1.5, ('m1','p2'):1, ('m1','p3'):2,
                 ('m2','p1'):3.5, ('m2','p2'):3, ('m2','p3'):4.5,
                 ('m3','p1'):4, ('m3','p2'):3, ('m3','p3'):4}
# e.g., machine 'm1' can produce 1.5 units of type 'p1' per hour but only 1 unit of type 'p2' per hour

Variables:

$$ h_{i,j}= \text{hours machine i spends producing product j} \quad \forall i\in MACHINES, j\in PRODUCTS $$

$$ m = \max\{\sum_{j\in PRODUCTS} h_{i,j}:\forall i\in MACHINES\} = \text{ maximum time any machine works in total} $$

Minimize the largest time any machine works across all products it produces:

Objective:
$$ \min m $$

Subject to constraints:
1. m is at least the time any machine works:
$$ m \geq \sum_{j\in PRODUCTS} h_{i,j} \quad \forall i\in MACHINES $$

2. demand constraints:
$$ \sum_{i\in MACHINES} h_{i,j}\cdot machine-speed[(i,j)] \geq d_j \quad \forall j\in PRODUCTS $$

3. non-negativity:
$$ h_{i,j} \geq 0 \quad \forall i \in \{machines\}, j \in \{products\}, m \geq 0$$

In [16]:
m = gp.Model("production_schedule")

time_spent = {}
# First add a decision variable capturing the time each machine spends on each product
for machine,p in machine_speed:
    time_spent[(machine,p)] =  m.addVar(lb= 0, name = machine+p)

# Next, add constraints to ensure we spend enough time (by respective machines) on each product so that enough units of each products are completed
##############################
# TODO: write your code here (complete the constraint(s) below)
for p in product_demand:
     m.addConstr(sum(time_spent[machine,p]*machine_speed[machine,p] for machine in ['m1', 'm2', 'm3']) >= product_demand[p])
##############################  
    
makespan = m.addVar(lb=0)   # makespan will be the objective, the maximum time any machine works in total. Notice that lb is the lower bound on a variable, so we do not allow the makespan to be negative

# Add a constraint that the makespan variable is no less than the time each machine spends individually
##############################
# TODO: complete the constraint(s) below
for machine in ['m1', 'm2', 'm3']:
    m.addConstr(makespan >= sum(time_spent[machine,p] for p in product_demand)) 
##############################  

##############################
# TODO: write your code here (write the objective function below)
m.setObjective(makespan, GRB.MINIMIZE)
##############################  

m.optimize()

print("The optimal schedule is:")
for x in time_spent:
    if time_spent[x].x>0:
        print("""Machine %s works on product %s for %g hours, producing %g units"""%(x[0], 
                                            x[1], time_spent[x].x, 
                                            machine_speed[x]*time_spent[x].x))

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.2.0 24C2101)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 6 rows, 10 columns and 21 nonzeros (Min)
Model fingerprint: 0xc264bf9c
Model has 1 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 4e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 5 rows, 9 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.250000e+02   0.000000e+00      0s
       4    7.3863636e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.386363636e+01
The optimal schedule is:
Machine m1 works on product p3 for 73.8636 hours, producing 147.727 units
Machine m2 works on produc

---
### **Problem 4: Supply Chain Optimization (75 points)**

This problem consists of 13 parts. Please refer to the "HW4 Supply Chain Optimization - Instructions.pdf" file for the problem background and other important information.

For the following problems, fill in the missing variables and constraints (i.e.,whenever there is a "{variable_name} = " or "{constraint_name} = " field you should fill in the line with your corresponding answer). If a question has a comment field, please provide an insightful comment on your results.

We'll re-import the relevant python packages for this problem. We will also import the data contained in the `HW4_data.py` file. Ensure that the file is in the same directory as this notebook and run the following cell:

In [21]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from HW4_data import *
%whos

Variable          Type         Data/Info
----------------------------------------
CE                ndarray      7: 7 elems, type `float64`, 56 bytes
CEU               ndarray      9x7: 63 elems, type `float64`, 504 bytes
GRB               type         <class 'gurobipy._grb.GRB'>
OEMD              ndarray      9x7: 63 elems, type `int64`, 504 bytes
PSCU              ndarray      9x7: 63 elems, type `float64`, 504 bytes
SCMT              ndarray      9x7: 63 elems, type `float64`, 504 bytes
UPC               ndarray      9: 9 elems, type `float64`, 72 bytes
buy               dict         n=9
c                 str          sodium
categories        tuplelist    ['calories', 'protein', 'fat', 'sodium']
cost              tupledict    {'hamburger': 2.69, 'chic<...> 0.89, 'ice cream': 1.99}
f                 str          ice cream
foods             tuplelist    ['hamburger', 'chicken', <...>ad', 'milk', 'ice cream']
gp                module       <module 'gurobipy' from '<...>es/gurobipy/__in

### Case background

IMT is consumer electronics company. Produces 2 products:
- 32" TV
- 42" TV

Production subconsraction to various design manufacturers.
- IMT pays for shipping from ODM to DC in Shanghai


IMT's customer has:
- Budget of CNY 3 billion 
- production and shpping og 920,000 units of 42" TVs and 530,000 units of 32" TVs to DC

IMT GOAL: configure most CO2 efficient supply chain that fulfills order within 3 billion budget


ODMS
- 7 different 
- OMD1, ODM2 produce both 32" and 42" TVs
- OMD3, ODM4, OMD5, ODM6, OMD7 produce only 42" TVs


Customer:
- 3 billion budget
- minimum order is 200,000 units to any ODM (i.e. each must order at least 200,000 units if we choose it)
- maximum order for either 32" or 42" TV is 600,000 units 
-- i.e.ODM1 can order 600,000 units of both

Several transport options:

Several transportation options are available to ship the TV sets from the ODMs to the DC: Regular Air,
Air Express, Road, Road LTL (less than truckload), Road Network, Rail, and Water. The distances from
the ODMs to the DC and the variuos shipping rates are included in the .py data file. ODM5 is located near
the DC and can only transport by Road, Road LTL and Road Network. ODM6 is located in South Korea
and shipping can only be done by Regular Air, Air Express, and Water. Across shipping modes, the rates of
carbon emission vary greatly from as high as 1.44 (Regular Air or Air Express) to 0.007 (Water) kilogram
per ton shipped per kilometer travelled. Each 32” LCD weighs 16.5 kg and each 42” LCD weighs 22 kg.
Shipping times vary from 2 days (Air Express) to 10 days (via Water). Based on historical information on
shipping times and customer order cycle times, the customer decided that to maintain satisfactory inventory
levels, IMT has to ship a minimum number of 42” and 32” LCD TVs according to the following criteria: for
42” (32”) TVs, a minimum of 46,000 (53,000) units must be shipped by Regular Air or Air express; 92,000
(79,500) must be shipped by Road or Road LTL or Road Network; and 138,000 (79,500) must be shipped
by Rail. There was no constraint on shipments via Water.
IMT would like to achieve the lowest possible carbon emissions while satisfying the above constraints. A
mathematical formulation of the optimization problem is included belo

&emsp; **1. Based on the information in the case, assign appropriate values to the constants in the following cell.** (5 pts)

In [ ]:
# Parameters


NameError: name 'HW_4data' is not defined

In [ ]:
# Total budget allocated for supply chain
TotalBudget = 30000000000000

# Total TV sets order by the customers from all ODMs by TV size

TotalUnits42 = 
TotalUnits32 = 

# Number of ODMs (treat ODMs making two different sizes as if they were separate ODMs, i.e., count them twice)
NumODM = 

# Number of transportation options
NumTran = 

# Minimum order size of 42" or 32" TV sets from any selected ODM
MinProd = 

# Maximum order size of 42" or 32" TV sets from any selected ODM
MaxProd = 

# Minimum number of 42" / 32" TV sets shipped by Regular Air or Air Express
MinAir42 = 
MinAir32 = 

# Minimum number of 42" / 32" TV sets shipped by Road, Road LTL, or Road Network
MinRoad42 = 
MinRoad32 = 

# Minimum number of 42" / 32" TV sets shipped by shipped by Rail
MinRail42 = 
MinRail32 = 

In [ ]:
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

# Initialize the model
m = gp.Model(env=env)

&emsp; **2. Add decision variables to the model.** (5 pts)

In [ ]:
### Decision Variables ###

# A vector of binary decision variables that denote which ODMs are selected
X = 

# Supply Chain Matrix
# Indicates how many TV sets will be produced by the selected ODMs and how they will be shipped to the distribution center
SCM =

&emsp; **3. Add minimum and maximum order size constraints for the ODMs.** (5 pts)

In [ ]:
### Constraints ###
# Total production for each ODM must be greater than the minimum order size if the ODM is selected
Const1a = 

# Total production for each ODM must be less than the maximum order size if the ODM is selected
Const1b = 

&emsp; **4. Add constraints for the minimum shipping requirements by transportation mode.** (5 pts)

In [ ]:
# Total number of TVs for each shipping mode must be greater than the Minimum Shipping Requirement
Const2a1 =                           # 42" Regualr Air or Air Express
Const2b1 =                           # 42" Road, Road LTL, or Road Network
Const2c1 =                           # 42" Rail
Const2a2 =                           # 32" Regular Air or Air Express
Const2b2 =                           # 32" Road, Road LTL, or Road Network
Const2c2 =                           # 32" Rail

&emsp; **5. Add any shipping method constraints for specific ODMs.** (5 pts)

In [ ]:
# ODM5 can't ship by Regular Air, Air Express, Rail, or Water (Hint: SCM [4, 1] corresponding to ODM5 shipping via Air Express)
Const3a =                  # Regular Air
Const3b =                  # Air Express
Const3c =                  # Rail
Const3d =                  # Water

# ODM6 can't ship by Road, Road LTL, Road Network, and Rail
Const4a =                  # Road
Const4b =                  # Road LTL
Const4c =                  # Road Network
Const4d =                  #Rail

&emsp; **6. Add your budget constraint.** (5 pts)

In [ ]:
# Total cost must be less than total budget, be sure to name this constraint 'Const5'!
Const5 = m.addConstr( # FILL- IN, name='Const5')

&emsp; **7. Add quantity constraints for each TV size (920,000 LCD 42” and 530,000 LCD 32”).** (5 pts)

In [ ]:
# Total units for each model must be equal to the desired quantities
Const6a =                 # 42" Total Units
Const6b =                 # 32" Total Units

&emsp; **8. Add an objective function minimizing CO2 emissions.** (5 pts)

In [ ]:
# Objective function: minimize CO2 emissions


&emsp; **9. Solve the model. What is the optimal objective function value (total CO2 emission)?** (5 pts)

In [ ]:
# Update and write the model
m.update() # Update model parameters
m.write("HW4.lp") # Write model to file

In [ ]:
# Solve


In [ ]:
# Print optimal objective function value


&emsp; **10. In the optimal solution, which ODMs are selected and how much is ordered from each ODM?** (5 pts)

In [ ]:
# Print the optimal decsion variable solutions
transport_names = ['Air', 'Express', 'Road', 'Road LTL', 'Road-Network', 'Rail', 'Water']
ODM_names = ['LCD 42" ODM1', 'LCD 42" ODM2', 'LCD 42" ODM3', 'LCD 42" ODM4', 'LCD 42" ODM5', 
             'LCD 42" ODM6', 'LCD 42" ODM7', 'LCD 32" ODM1', 'LCD 32" ODM2']

pd.DataFrame(SCM.getAttr('x'), index= , columns= ) # You must fill in the values for index and columns

&emsp; **11. In the optimal solution, check whether the budget constraint is binding (i.e., is the CNY 3 billion budget fully used)?** (5 pts)

In [ ]:
# Calculate used budget

**Comment**: [YOUR COMMENT HERE]


&emsp; **12. Now imagine your budget is increased by 10% to CNY 3.3 billion. How much and by what percentage is the CO2 emission reduced? How does your optimal sourcing decisions change? Do you see a shift in the modes of transportation compared to the case with CNY 3 billion budget? Is this surprising?** (10 pts)

In [ ]:
newTotalBudget = 
# Hint: You can change the value of the total budget with this command, by changing the right
# hand side of Constraint 5, be sure constraint 5 is named 'Const5'
m.getConstrByName('Const5').setAttr('RHS', newTotalBudget)

# Re-solve
m.update()
m.optimize()

# Print optimal objective function value
print("\nObjective value: ", m.getAttr("ObjVal"))

pd.DataFrame(SCM.getAttr('x'), index=ODM_names, columns=transport_names)

**Comment**: [YOUR COMMENT HERE]

&emsp; **13. Imagine your budget increases by the following percentages: 0%, 2%, 4%, 6%, 10%, 12%, and 14%. For each of these budgets, find the CO2 emission under the optimal solution (re-run your optimization model for each of these budgets). Plot these results where the x-axis in the %Budget Increase and the y-axis is the CO2 Emission. Show the chart. What do you observe?** (10 pts)

In [ ]:
# sensitivity table
budget_sensitivity = [] # FILL-IN

# We will create a vector of emissions by looping through budget_sensitivity
emissions = []
for sensitivity in budget_sensitivity:
    newTotalBudget = # FILL IN
    # Hint: You can change the value of the total budget with this command
    m.getConstrByName('Const5').setAttr('RHS', newTotalBudget)

    # Re-solve
    m.update()
    m.optimize()

    # Print optimal objective function value
    emissions.append(m.getAttr("ObjVal"))
    
print(emissions)

In [ ]:
# Graph results
# FILL-IN

**Comment**: [YOUR COMMENT HERE]

********************************************************************************************************************************
### End of Homework 4